In [3]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot  as plt
import seaborn as sn
%matplotlib inline
import pandas as pd
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import LabelEncoder
from tabulate import tabulate
import csv
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso

In [4]:
df = pd.read_csv("/home/growlt199/Downloads/flight_price_prediction/data/flight_processed_data.csv",index_col=0)

In [5]:
df.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,SpiceJet,SG-8709,Delhi,Evening,0,Night,Mumbai,0,2.17,1,5953
1,SpiceJet,SG-8157,Delhi,Early_Morning,0,Morning,Mumbai,0,2.33,1,5953
2,AirAsia,I5-764,Delhi,Early_Morning,0,Early_Morning,Mumbai,0,2.17,1,5956
3,Vistara,UK-995,Delhi,Morning,0,Afternoon,Mumbai,0,2.25,1,5955
4,Vistara,UK-963,Delhi,Morning,0,Morning,Mumbai,0,2.33,1,5955


In [6]:
#used labelecoding in flight column
label_encoder = LabelEncoder()
df['flight'] = label_encoder.fit_transform(df['flight'])

In [7]:
df.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,SpiceJet,1408,Delhi,Evening,0,Night,Mumbai,0,2.17,1,5953
1,SpiceJet,1387,Delhi,Early_Morning,0,Morning,Mumbai,0,2.33,1,5953
2,AirAsia,1213,Delhi,Early_Morning,0,Early_Morning,Mumbai,0,2.17,1,5956
3,Vistara,1559,Delhi,Morning,0,Afternoon,Mumbai,0,2.25,1,5955
4,Vistara,1549,Delhi,Morning,0,Morning,Mumbai,0,2.33,1,5955


In [8]:
def data_to_one_hot(df):
    # get dummy variables for multiple columns
    dummy_cols = ['airline', 'departure_time','arrival_time','source_city','destination_city']
    df1_dummy = pd.get_dummies(df[dummy_cols])

    # concatenate original DataFrame with dummy variable DataFrame
    df1_concat = pd.concat([df.drop(dummy_cols, axis=1), df1_dummy], axis=1)
    
    # display result
    return df1_concat

def data_label_encoding(df):
    from sklearn.preprocessing import LabelEncoder

    # Create an instance of LabelEncoder
    label_encoder = LabelEncoder()

    # Specify the list of categorical columns to encode
    categorical_columns = ['airline', 'departure_time','arrival_time','source_city','destination_city']

    # Apply label encoding to each categorical column
    for column in categorical_columns:
        df[column] = label_encoder.fit_transform(df[column])
    
    return df

In [9]:
def evaluation(model_name,enc,y_test,predictions,train_score,test_score):
    data = [
    ['model name', model_name],
    ['encoding', enc],
    ['MAE', metrics.mean_absolute_error(y_test, predictions)],
    ['MSE', metrics.mean_squared_error(y_test, predictions)],
    ['RMSE', np.sqrt(metrics.mean_squared_error(y_test, predictions))],
    ['r2', r2_score(y_test, predictions)],
    ['explained_variance_score', metrics.explained_variance_score(y_test, predictions)],
    ['max_error', metrics.max_error(y_test, predictions)],
    ['mean_absolute_percentage_error', metrics.mean_absolute_percentage_error(y_test, predictions)],
    ['mean_squared_log_error', metrics.mean_squared_log_error(y_test, predictions) if all(x >= 0 for x in y_test)
     and all(y >= 0 for y in predictions) else 'N/A'],
    ['mean_squared_log_error(squared=False)', metrics.mean_squared_log_error(y_test, predictions, squared=False)
     if all(x >= 0 for x in y_test) and all(y >= 0 for y in predictions) else 'N/A'],
    ['test score',test_score],
    ['train score',train_score]
    ]

    
    # Define the headers
    headers = ["metrics", "value"]

    # Display the data in a tabular format
    table = tabulate(data, headers, tablefmt="grid")
    print(table)
    
    transposed_data = list(zip(*data))
    
    return transposed_data[1]

In [10]:
def csv_dump(matrix,filename):
  
    with open(filename, 'a', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(matrix)

In [11]:
def model_random_forest(x_train,y_train,x_test,y_test,enc,filename):

    import time
    start_time = time.time()
    
    print(f'model name :- random forest regression && encoding :-{enc}')
    #model 
    model = RandomForestRegressor(random_state=123)

    #training model input as x_train and y_tarin
    model.fit(x_train,y_train)

    #model is creted show give x test data as input and retuen predictd data 
    predictions = model.predict(x_test)
    
    #scores
    train_score = model.score(x_train,y_train)
    test_score = model.score(x_test,y_test)
    
    # evaluation
    eva_matrix = evaluation('random forest',enc,y_test,predictions,train_score,test_score)
    
    #dump in csv 
    csv_dump(list(eva_matrix),filename)
    
    # Code or function to measure execution time
    end_time = time.time()
    execution_time = end_time - start_time
    print("Execution time:", execution_time, "seconds")

In [12]:
def model_linear_regression(x_train,y_train,x_test,y_test,enc,filename):

    import time
    start_time = time.time()
    
    print(f'model name :- linear regression && encoding :-{enc}')
    #model 
    model = LinearRegression()

    #training model input as x_train and y_tarin
    model.fit(x_train,y_train)

    #model is creted show give x test data as input and retuen predictd data 
    predictions = model.predict(x_test)
       
    #scores
    train_score = model.score(x_train,y_train)
    test_score = model.score(x_test,y_test)
    
    # evaluation
    eva_matrix = evaluation('linear regression',enc,y_test,predictions,train_score,test_score)
    
    #dump in csv 
    csv_dump(list(eva_matrix),filename)
    
    # Code or function to measure execution time
    end_time = time.time()
    execution_time = end_time - start_time
    print("Execution time:", execution_time, "seconds")

In [13]:
def model_lasso(x_train,y_train,x_test,y_test,enc,filename):
    
    import time
    start_time = time.time()
    print(f'model name :- lasso regression && encoding :-{enc}')
    # Create a Lasso regression model
    lasso = Lasso(alpha=0.1
                 ) 

    # Fit the model to the training data
    lasso.fit(x_train, y_train)

    # Make predictions on the test data
    predictions = lasso.predict(x_test)
    
    # evaluation
    eva_matrix = evaluation('lasso',enc,y_test,predictions,'N/A','N/A')
    
    #dump in csv 
    csv_dump(list(eva_matrix),filename)
    
    # Code or function to measure execution time
    end_time = time.time()
    execution_time = end_time - start_time
    print("Execution time:", execution_time, "seconds")

In [14]:
def model_gradiant_boosting(x_train,y_train,x_test,y_test,enc,filename):
    import time

    start_time = time.time()

    #import model from sklearn
    from sklearn.ensemble import GradientBoostingRegressor
    print(f'model name :- gradiant boosting regression && encoding:- {enc}')

    model = GradientBoostingRegressor(random_state=123)

    #training model input as x_train and y_tarin
    model.fit(x_train,y_train)
    
    #model is creted show give x test data as input and retuen predictd data 
    predictions = model.predict(x_test)
    
    #scores
    train_score = model.score(x_train,y_train)
    test_score = model.score(x_test,y_test)

    # evaluation
    eva_matrix = evaluation('gradiant boosting',enc,y_test,predictions,train_score,test_score)
    
    #dump in csv 
    csv_dump(list(eva_matrix),filename)
    
    # Code or function to measure execution time
    end_time = time.time()
    execution_time = end_time - start_time
    print("Execution time:", execution_time, "seconds")

In [15]:
def model_decision_tree_regression(x_train,y_train,x_test,y_test,enc,filename):
    import time

    start_time = time.time()

    print(f'model name :- DecisionTreeRegressor && encoding:- {enc}')

    model = DecisionTreeRegressor(random_state=123)

    #training model input as x_train and y_tarin
    model.fit(x_train,y_train)

    #model is creted show give x test data as input and retuen predictd data 
    predictions = model.predict(x_test)
    
    #scores
    train_score = model.score(x_train,y_train)
    test_score = model.score(x_test,y_test)

    # evaluation
    eva_matrix = evaluation('DecisionTreeRegressor',enc,y_test,predictions,train_score,test_score)
    
    #dump in csv 
    csv_dump(list(eva_matrix),filename)
    
    # Code or function to measure execution time
    end_time = time.time()
    execution_time = end_time - start_time
    print("Execution time:", execution_time, "seconds")

In [16]:
df.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,SpiceJet,1408,Delhi,Evening,0,Night,Mumbai,0,2.17,1,5953
1,SpiceJet,1387,Delhi,Early_Morning,0,Morning,Mumbai,0,2.33,1,5953
2,AirAsia,1213,Delhi,Early_Morning,0,Early_Morning,Mumbai,0,2.17,1,5956
3,Vistara,1559,Delhi,Morning,0,Afternoon,Mumbai,0,2.25,1,5955
4,Vistara,1549,Delhi,Morning,0,Morning,Mumbai,0,2.33,1,5955


In [17]:
list_models = ['model_random_forest','model_linear_regression','model_gradiant_boosting'
               ,'model_lasso','model_decision_tree_regression']
encoding = ['data_to_one_hot','data_label_encoding']

In [18]:
import os
import sys

#main script
filename = '/home/growlt199/Downloads/flight_price_prediction/model_training/model_weightage/evaluation_dif_models.csv'

# Check if the file exists
if not os.path.exists(filename):
    print(f"The file '{filename}' does not exist.")
    
    with open(filename, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(['model name ','encoding',
                        'mae','mse','rmse','r2',
                        'explained_variance_score', 'max_error',
                        'mean_absolute_percentage_error', 'mean_squared_log_error', 
                        'mean_squared_log_error(squared=False)',
                        'test_score','train_score']) 


    for model in list_models:
        for enc in encoding:
            model_fuc = globals().get(model)
            enc_fuc = globals().get(enc)

            #encoding
            df1 = enc_fuc(df.copy())

            #create x and y where x are features for the model and y as target
            x = df1.drop('price',axis=1)
            y = df1['price']

            #split data
            x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state =123, shuffle=True)

            model_fuc(x_train,y_train,x_test,y_test,enc,filename)

else:
    # Continue with the rest of your code if the file exists
    print(f"The file  exists. Continue with the program.")
    # Rest of your code...



The file  exists. Continue with the program.


In [19]:
df.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
0,SpiceJet,1408,Delhi,Evening,0,Night,Mumbai,0,2.17,1,5953
1,SpiceJet,1387,Delhi,Early_Morning,0,Morning,Mumbai,0,2.33,1,5953
2,AirAsia,1213,Delhi,Early_Morning,0,Early_Morning,Mumbai,0,2.17,1,5956
3,Vistara,1559,Delhi,Morning,0,Afternoon,Mumbai,0,2.25,1,5955
4,Vistara,1549,Delhi,Morning,0,Morning,Mumbai,0,2.33,1,5955


# comparing different model accuracy and other evaluation matrix

In [20]:
results = pd.read_csv('/home/growlt199/Downloads/flight_price_prediction/model_training/model_weightage/evaluation_dif_models.csv')

In [21]:
results.head()

,model name,encoding,mae,mse,rmse,r2,explained_variance_score,max_error,mean_absolute_percentage_error,mean_squared_log_error,mean_squared_log_error(squared=False),test_score,train_score
0,random forest,data_to_one_hot,810.867744,4.516476e+06,2125.200336,0.991057,0.991059,32685.600000,0.055818,0.013985,0.118257,0.991057,0.998671
1,random forest,data_label_encoding,830.815782,4.755086e+06,2180.615930,0.990585,0.990587,32272.700000,0.056669,0.014259,0.119412,0.990585,0.998622
2,linear regression,data_to_one_hot,4438.980054,4.378273e+07,6616.852251,0.913310,0.913310,39022.873146,0.425319,NaN,NaN,0.913310,0.913947
3,linear regression,data_label_encoding,4464.952165,4.548017e+07,6743.898979,0.909949,0.909950,40402.968318,0.417882,NaN,NaN,0.909949,0.910407
4,gradiant boosting,data_to_one_hot,2713.243155,2.004607e+07,4477.283235,0.960309,0.960309,46688.591072,0.202139,0.066888,0.258628,0.960309,0.960794


In [22]:
results.sort_values(['mae','r2'], ascending=True)

,model name,encoding,mae,mse,rmse,r2,explained_variance_score,max_error,mean_absolute_percentage_error,mean_squared_log_error,mean_squared_log_error(squared=False),test_score,train_score
0,random forest,data_to_one_hot,810.867744,4.516476e+06,2125.200336,0.991057,0.991059,32685.600000,0.055818,0.013985,0.118257,0.991057,0.998671
8,DecisionTreeRegressor,data_to_one_hot,825.533506,7.130639e+06,2670.325565,0.985881,0.985882,31920.000000,0.058325,0.022857,0.151187,0.985881,0.999913
1,random forest,data_label_encoding,830.815782,4.755086e+06,2180.615930,0.990585,0.990587,32272.700000,0.056669,0.014259,0.119412,0.990585,0.998622
9,DecisionTreeRegressor,data_label_encoding,848.536437,7.563127e+06,2750.113923,0.985025,0.985028,31428.000000,0.059520,0.023431,0.153070,0.985025,0.999913
4,gradiant boosting,data_to_one_hot,2713.243155,2.004607e+07,4477.283235,0.960309,0.960309,46688.591072,0.202139,0.066888,0.258628,0.960309,0.960794
5,gradiant boosting,data_label_encoding,2734.910234,2.030713e+07,4506.343789,0.959792,0.959792,47179.018784,0.202282,0.066548,0.257969,0.959792,0.960351
6,lasso,data_to_one_hot,4438.857353,4.378261e+07,6616.843103,0.913310,0.913310,39022.053621,0.425279,NaN,NaN,NaN,NaN
2,linear regression,data_to_one_hot,4438.980054,4.378273e+07,6616.852251,0.913310,0.913310,39022.873146,0.425319,NaN,NaN,0.913310,0.913947
7,lasso,data_label_encoding,4464.895997,4.548017e+07,6743.898569,0.909949,0.909950,40401.816528,0.417860,NaN,NaN,NaN,NaN
3,linear regression,data_label_encoding,4464.952165,4.548017e+07,6743.898979,0.909949,0.909950,40402.968318,0.417882,NaN,NaN,0.909949,0.910407


# now random forest regression is base model 

In [23]:
# Take a sample of 10,000 rows from the DataFrame
df2 = df.sample(n=10000, random_state=42)

In [24]:
df2.head()

,airline,flight,source_city,departure_time,stops,arrival_time,destination_city,class,duration,days_left,price
6048,Indigo,110,Delhi,Afternoon,1,Evening,Mumbai,0,3.83,31,3631
228725,Vistara,1555,Mumbai,Night,0,Late_Night,Delhi,1,2.17,38,22140
226756,Air_India,815,Mumbai,Morning,1,Evening,Delhi,1,10.83,19,50957
225700,Vistara,1435,Mumbai,Morning,1,Evening,Delhi,1,5.25,9,37374
15828,Indigo,131,Delhi,Afternoon,1,Evening,Bangalore,0,5.17,30,4007


In [ ]:
import copy
all_rmse=[]
all_degree=[]
from sklearn.preprocessing import PolynomialFeatures
from sklearn import linear_model

for i in range(1,6):
    for enc in encoding:
        enc_fuc = globals().get(enc)

        #encoding
        df1 = enc_fuc(df2.copy())

        #create x and y where x are features for the model and y as target
        x = df1.drop('price',axis=1)
        y = df1['price']

        #split data
        x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.2, random_state =123, shuffle=True)
        
        poly_func=PolynomialFeatures(degree=i)
        x_train_poly=poly_func.fit_transform(x_train)
            
        # train_x_poly
        new_lm=linear_model.LinearRegression()
        train_y_=new_lm.fit(x_train_poly,y_train)
        x_test_poly=poly_func.fit_transform(x_test)
        y_predict=new_lm.predict(x_test_poly)
        rmse=np.sqrt(np.mean((y_predict-y_test)**2))
        #print("\t{0:5}{1:7}".format(i,''),end='')
        print(f"{rmse}")
        # print(i)
#         latest_model=copy.deepcopy(new_lm)
#         latest_degree=copy.deepcopy(i)
#         latest_rmse=copy.deepcopy(rmse)
#         all_rmse.append(rmse)
#         all_degree.append(i)
        

6598.349552383732
6723.538010927085
302209.2829093978
5097.982283526015
231263.4846184413
4692.089434833619
